In [3]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [4]:
def closeness_score(x_i, x_j, t):
  return np.exp((-1)*np.linalg.norm(x_i-x_j)/t)


In [5]:
#demonstration of closeness_score
x_i = np.array([1, 0])
x_j = np.array([0, 1])
closeness_score(x_i, x_j, 2) #ok so this seems to work as expected

np.float64(0.4930686913952398)

In [21]:
def calculate_lp_score(X, k, t, r, distance_matrix='standard'):
  #have: n samples, each with m features.

  m, n = X.shape

  #Step 1: generate nearest neighbor graph
  nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(X)
  distances, indices = nbrs.kneighbors(X)

  S = np.zeros([m, m])

  #Step 2: generate matrix S
  if distance_matrix == 'standard':
    for edge in indices:
      a = edge[0]
      b = edge[1]
      S[a][b] = closeness_score(a, b, t)
      S[b][a] = closeness_score(a, b, t)
      #figure out if there is a way to yassify this with numpy, probably

    #Generate L
    # D_vector = S * 1 (row sums of S)
    D_vector = np.sum(S, axis=1)
    D = np.diag(D_vector)
    L = D - S

    # Safety check for all-zero D
    d_sum = np.sum(D_vector)
    if d_sum == 0:
         return np.zeros(n)


    #Step 3: remove the mean for all features
    # print(X.T.shape, D_vector.shape)
    M = (X.T @ D_vector[:, np.newaxis])/ d_sum

    F_t = X - np.outer(np.ones(m), M)
    #i.e., F-tilde, take outer product with mean vector for mean matrix

    #Step 4: calculate vectorized L_r for all r
    num_matrix = F_t.T @ L @ F_t
    num_vector = np.diag(num_matrix)

    denom_matrix = F_t.T @ D @ F_t
    denom_vector = np.diag(denom_matrix)

    L_r_scores = np.divide(num_vector, denom_vector,
                           out=np.zeros_like(num_vector, dtype=float),
                           where=denom_vector != 0)
    return L_r_scores

In [19]:
X = np.random.rand(3,4)
X

array([[0.58087581, 0.26396591, 0.71898059, 0.23767985],
       [0.01193672, 0.17690365, 0.50816987, 0.59678009],
       [0.48986907, 0.30124805, 0.44908803, 0.88877735]])

In [22]:
print(calculate_lp_score(X, 2, 2, 2))

[1.98511066 1.94144176 1.13646048 1.00528277]
